In [10]:
#globでファイル数を求めてforを回す準備
import os
import pandas as pd

path = "./resultFCNNx"
filelist = os.listdir(path)
subject_count = 11
print(filelist)

['0_report.csv', '1_report.csv', '2_report.csv', '3_report.csv', '4_report.csv', '5_report.csv', '6_report.csv', '7_report.csv', '8_report.csv', '9_report.csv', '10_report.csv']


In [11]:
#precision_1みたいにして列名が回数ごとでかぶらないようにする
df_list = []
for i in range(len(filelist)):
    df_csv = pd.read_csv(
        f"{path}/{i}_report.csv", sep=",",header=0
        ,names=["被験者","precision_"+str(i),"recall_"+str(i),"f-score_"+str(i),"support"]
        ,index_col="被験者",skiprows=[subject_count,subject_count+1,subject_count+2]
    )
    df_csv= df_csv.drop("support",axis=1)
    print(df_csv.index)
    for j in range(subject_count-1):
        df_csv = df_csv.rename(index={df_csv.index[j]:chr(df_csv.index[j]+65)})

    df_mean = [df_csv["precision_"+str(i)].mean(),df_csv["recall_"+str(i)].mean(),df_csv["f-score_"+str(i)].mean()]
    df_std = [df_csv["precision_"+str(i)].std(),df_csv["recall_"+str(i)].std(),df_csv["f-score_"+str(i)].std()]
    df_csv.loc[subject_count]= df_mean
    df_csv.loc[subject_count+1]=df_std
    df_csv = df_csv.rename({subject_count:"平均"})
    df_csv = df_csv.rename({subject_count+1:"標準偏差"})
    df_list.append(df_csv)
    #display(df_csv)

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64', name='被験者')
Int64Index([0, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64', name='被験者')
Int64Index([0, 1, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64', name='被験者')
Int64Index([0, 1, 2, 4, 5, 6, 7, 8, 9, 10], dtype='int64', name='被験者')
Int64Index([0, 1, 2, 3, 5, 6, 7, 8, 9, 10], dtype='int64', name='被験者')
Int64Index([0, 1, 2, 3, 4, 6, 7, 8, 9, 10], dtype='int64', name='被験者')
Int64Index([0, 1, 2, 3, 4, 5, 7, 8, 9, 10], dtype='int64', name='被験者')
Int64Index([0, 1, 2, 3, 4, 5, 6, 8, 9, 10], dtype='int64', name='被験者')
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 9, 10], dtype='int64', name='被験者')
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 10], dtype='int64', name='被験者')
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64', name='被験者')


In [12]:
#len(df_list)回分を結合でまとめる
precision_column = []
recall_column = []
fscore_column = []

for i in range(len(df_list)):
    if i == 0:
        df_subject = df_list[0]
    else:
        df_subject = pd.merge(df_subject,df_list[i], how='outer', left_index=True, right_index=True)
    
    precision_column.append(f"precision_{i}")
    recall_column.append(f"recall_{i}")
    fscore_column.append(f"f-score_{i}")

columns = [precision_column,recall_column,fscore_column]

In [13]:
result = []

result.append(df_subject.index.to_list())

for column in columns:
    df_subject_mean = df_subject[column].mean(axis='columns')
    df_subject_std = df_subject[column].std(axis='columns')
    result.append(df_subject_mean.to_list())
    result.append(df_subject_std.to_list())

df_result = pd.DataFrame(result).T
for i in range(len(df_result.columns)-1):
    df_result[i+1] = df_result[i+1].astype(float)

df_result = df_result.round(3)

for i in range(len(df_result.columns)-1):
    df_result[i+1] = df_result[i+1].astype(str)

final_result =[]
final_result.append(df_result[1]+""+"("+df_result[2]+")")
final_result.append(df_result[3]+""+"("+df_result[4]+")")
final_result.append(df_result[5]+""+"("+df_result[6]+")")

final_result = pd.DataFrame(final_result).T
final_result.columns = ["Precision","Recall","F-score"]
for j in range(len(filelist)):
        final_result = final_result.rename(index={final_result.index[j]:chr(final_result.index[j]+65)})
final_result = final_result.rename({subject_count:"平均"})
final_result = final_result.rename({subject_count+1:"標準偏差"})
display(final_result)

,Precision,Recall,F-score
A,0.991(0.005),0.998(0.002),0.995(0.003)
B,0.981(0.012),0.995(0.004),0.988(0.006)
C,0.993(0.005),0.992(0.004),0.992(0.004)
D,0.99(0.006),0.987(0.013),0.989(0.008)
E,0.997(0.004),0.997(0.002),0.997(0.002)
F,0.998(0.002),1.0(0.0),0.999(0.001)
G,0.986(0.006),0.979(0.009),0.982(0.006)
H,0.997(0.003),0.995(0.004),0.996(0.002)
I,0.995(0.002),0.994(0.005),0.995(0.003)
J,0.993(0.005),0.981(0.004),0.987(0.003)
